https://github.com/utkuozbulak/pytorch-cnn-visualizations
https://towardsdatascience.com/how-to-visualize-convolutional-features-in-40-lines-of-code-70b7d87b0030

In [1]:
import h5py
import os
from os.path import join
from glob import glob
from scipy.io import loadmat
import cv2
import torch
from torch.autograd import Variable
from torch.autograd import grad
import numpy as np
import matplotlib.pylab as plt
import csv
from time import time

In [9]:
torch.cuda.get_device_name(device=None)

'GeForce GTX 1060 6GB'

In [2]:
import torchvision

In [3]:
alexnet=torchvision.models.alexnet(pretrained=True)

In [177]:
net = torchvision.models.vgg16(pretrained=True)

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to C:\Users\binxu/.cache\torch\checkpoints\vgg16-397923af.pth
94.5%

In [180]:
net

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [4]:
from torchvision import transforms
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])  # Note without normalization, the
denormalize = transforms.Normalize(
    mean=[-0.485/0.229, -0.456/0.224, -0.406/0.225],
    std=[1/0.229, 1/0.224, 1/0.225])

In [5]:
class SaveFeatures():
    def __init__(self, module):
        self.hook = module.register_forward_hook(self.hook_fn)
    def hook_fn(self, module, input, output):
        self.features = output#torch.tensor(output,requires_grad=True).cuda()
    def close(self):
        self.hook.remove()

def val_tfms(img_np):
    img = torch.from_numpy(img_np.astype(np.float32)).permute(2, 0, 1)
    nimg = normalize(img).unsqueeze(0).cuda()
    return nimg

def val_detfms(img_tsr):
    img = denormalize(img_tsr.squeeze()).permute(1,2,0)
    return img.detach().cpu().numpy()

In [10]:
MAX_TRIAL = 100
class FilterVisualizer():
    def __init__(self, model, size=56, upscaling_steps=12, upscaling_factor=1.2):
        self.size, self.upscaling_steps, self.upscaling_factor = size, upscaling_steps, upscaling_factor
        self.model = model #alexnet.features.cuda().eval()
        # set_trainable(self.model, False)
        for param in self.model.parameters():
            param.requires_grad = False

    def visualize(self, layer, filter, lr=0.05, opt_steps=20, blur=None):
        sz = self.size
        img = np.uint8(np.random.uniform(50, 250, (sz, sz, 3))) / 255  # generate random image
        activations = SaveFeatures(list(self.model.children())[layer])  # register hook

        for _ in range(self.upscaling_steps):  # scale the image up upscaling_steps times
            # train_tfms, val_tfms = tfms_from_model(vgg16, sz)
            img_var = Variable(val_tfms(img), requires_grad=True) # convert image to Variable that requires grad
            optimizer = torch.optim.Adam([img_var], lr=lr, weight_decay=1e-6)
            for n in range(MAX_TRIAL):
                optimizer.zero_grad()
                self.model(img_var)
                loss = -activations.features[0, filter].mean()
                loss.backward()
                if img_var.grad.norm()<1E-6:
                    img_var = Variable(val_tfms(img + np.random.randn(*img.shape)), requires_grad=True) # convert image to Variable that requires grad
                    optimizer = torch.optim.Adam([img_var], lr=lr, weight_decay=1e-6)
                    print("Optimizer restart")
                else:
                    break
            for n in range(opt_steps):  # optimize pixel values for opt_steps times
                optimizer.zero_grad()
                self.model(img_var)
                loss = -activations.features[0, filter].mean()
                loss.backward()
                optimizer.step()
                print(loss.data.cpu())
            img = val_detfms(img_var.data.cpu())
            self.output = img
            plt.figure(figsize=[8,8])
            plt.imshow(FVis.output)
            plt.show()
            sz = int(self.upscaling_factor * sz)  # calculate new image size
            img = cv2.resize(img, (sz, sz), interpolation=cv2.INTER_CUBIC)  # scale image up
            if blur is not None: img = cv2.blur(img, (blur, blur))  # blur image to reduce high frequency patterns
        self.save(layer, filter)
        activations.close()

    def save(self, layer, filter):
        plt.imsave("layer_" + str(layer) + "_filter_" + str(filter) + ".jpg", np.clip(self.output, 0, 1))

#%%

In [12]:
feat = alexnet.features.cuda().eval()

RuntimeError: CUDA error: all CUDA-capable devices are busy or unavailable

In [ ]:
feat = net.features.cuda().eval()
FVis = FilterVisualizer(feat, size=227, upscaling_steps=2, upscaling_factor=1.2)
FVis.visualize(14, 20, blur=10, opt_steps=20)
plt.figure(figsize=[8,8])
plt.imshow(FVis.output)
plt.show()

In [ ]:
net = alexnet.features[:]

In [ ]:
sz = 224
lr = 0.1
opt_steps = 100
upscaling_factor = 1.2
filter = 10
loss_arr = []
activations = SaveFeatures(list(alexnet.features.children())[8])  # register hook

img = np.uint8(np.random.uniform(150, 180, (sz, sz, 3))) / 255
img_var = Variable(val_tfms(img), requires_grad=True) # convert image to Variable that requires grad
optimizer = torch.optim.Adam([img_var], lr=lr, weight_decay=1e-6)
for n in range(opt_steps):  # optimize pixel values for opt_steps times
    optimizer.zero_grad()
    alexnet.features(img_var)
    loss = -activations.features[0, filter].mean()
    loss.backward()
    optimizer.step()
    loss_arr.append(loss.data.cpu())
img = val_detfms(img_var.data.cpu()).numpy()
output = img
sz = int(upscaling_factor * sz)  # calculate new image size
img = cv2.resize(img, (sz, sz), interpolation=cv2.INTER_CUBIC)

In [ ]:
alexnet.features(img_var)
loss = -activations.features[0, filter].mean()

In [ ]:
FVis.output[:,:,0].mean()

In [ ]:
img_np = np.random.rand(5,5,3)
img_tsr = val_tfms(img_np)
img_out = val_detfms(img_tsr)

## Printing Version of FilterVisualization

In [ ]:
# def np2tensor(image,dtype):
#     "Convert np.array (sz,sz,3) to tensor (1,3,sz,sz), imagenet normalized"

#     a = np.asarray(image)
#     if a.ndim==2 : a = np.expand_dims(a,2)
#     a = np.transpose(a, (1, 0, 2))
#     a = np.transpose(a, (2, 1, 0))
    
#     #Imagenet norm
#     mean=np.array([0.485, 0.456, 0.406])[...,np.newaxis,np.newaxis]
#     std = np.array([0.229, 0.224, 0.225])[...,np.newaxis,np.newaxis]
#     a = (a-mean)/std
#     a = np.expand_dims(a,0)
#     return torch.from_numpy(a.astype(dtype, copy=False) )

# def tensor2np(img_tensor):
#     "Convert tensor (1,3,sz,sz) back to np.array (sz,sz,3), imagenet DEnormalized"
#     a = np.squeeze(to_np(img_tensor))
    
#     mean=np.array([0.485, 0.456, 0.406])[...,np.newaxis,np.newaxis]
#     std = np.array([0.229, 0.224, 0.225])[...,np.newaxis,np.newaxis]
#     a = a*std + mean
#     return np.transpose(a, (1,2,0))

class FilterVisualizer():
    def __init__(self,model):
        self.model = model
        self.weights = None

    def visualize(self, sz, layer, filter, weights=None, 
                  upscaling_steps=12, upscaling_factor=1.2, lr=0.1, opt_steps=20, blur=None, print_losses=False):
        '''Add weights to support visualize combination of channels'''
        if weights is not None:
            assert len(weights) == len(filter)
            self.weights = torch.tensor(weights,dtype=torch.float,device='cuda')
        img = (np.random.random((sz,sz, 3)) * 20 + 128.)/255 # value b/t 0 and 1        
        activations = SaveFeatures(layer)  # register hook

        for i in range(upscaling_steps):  
            # convert np to tensor + channel first + new axis, and apply imagenet norm
            img_tensor = val_tfms(img)#,np.float32)
            img_tensor = img_tensor.cuda()
            img_tensor.requires_grad_();
            if not img_tensor.grad is None:
                img_tensor.grad.zero_(); 
            
            
            optimizer = torch.optim.Adam([img_tensor], lr=0.1, weight_decay=1e-6)
            if i > upscaling_steps/2:
                opt_steps_ = int(opt_steps*1.3)
            else:
                opt_steps_ = opt_steps
            for n in range(opt_steps_):  # optimize pixel values for opt_steps times
                optimizer.zero_grad()
                _=self.model(img_tensor)
                if weights is None:
                    loss = -1*activations.features[0, filter].mean()
                else: 
                    loss = -1*torch.einsum("ijk,i->jk", activations.features[0, filter], self.weights).mean()
                if print_losses:
                    if i%3==0 and n%5==0:
                        print(f'{i} - {n} - {float(-loss)}')
                loss.backward()
                optimizer.step()
            
            # convert tensor back to np
            img = val_detfms(img_tensor)
            self.output = img
            sz = int(upscaling_factor * sz)  # calculate new image size
#             print(f'Upscale img to: {sz}')
            img = cv2.resize(img, (sz, sz), interpolation = cv2.INTER_CUBIC)  # scale image up
            if blur is not None: img = cv2.blur(img,(blur,blur))  # blur image to reduce high frequency patterns
                
        activations.close()
        return np.clip(self.output, 0, 1)
    
    def get_transformed_img(self,img,sz):
        '''
        Scale up/down img to sz. Channel last (same as input)
        image: np.array [sz,sz,3], already divided by 255"    
        '''
        return cv2.resize(img, (sz, sz), interpolation = cv2.INTER_CUBIC)
    
    def most_activated(self, img, layer):
        '''
        image: np.array [sz,sz,3], already divided by 255"    
        '''
        img = cv2.resize(img, (224,224), interpolation = cv2.INTER_CUBIC)
        activations = SaveFeatures(layer)
        img_tensor = val_tfms(img)#,np.float32)
        img_tensor = img_tensor.cuda()
        
        _=self.model(img_tensor)
        mean_act = [np.squeeze(to_np(activations.features[0,i].mean())) for i in range(activations.features.shape[1])]
        activations.close()
        return mean_act

In [ ]:
feat = alexnet.features.cuda().eval()
FVis = FilterVisualizer(feat)

In [ ]:
img = FVis.visualize(sz=227, layer=feat[8], filter=[1,5,3,10], weights=[1,3,1,7], blur=10, opt_steps=20, upscaling_steps=3, upscaling_factor=1.2, print_losses=True)
plt.figure(figsize=[8,8])
plt.imshow(FVis.output)
plt.show()

## Using GAN to visualize Convolutiona Layers

In [ ]:
BGR_mean = torch.tensor([104.0, 117.0, 123.0])
BGR_mean = torch.reshape(BGR_mean, (1, 3, 1, 1))
def visualize(G, code):
    """Do the De-caffe transform (Validated)"""
    code = code.reshape(-1, 4096).astype(np.float32)
    blobs = G(torch.from_numpy(code))
    out_img = blobs['deconv0']  # get raw output image from GAN
    clamp_out_img = torch.clamp(out_img + BGR_mean, 0, 255)
    vis_img = clamp_out_img[:, [2, 1, 0], :, :].permute([2, 3, 1, 0]).squeeze() / 255
    return vis_img

def visualize_for_torchnet(G, code):
    """Do the De-caffe transform (Validated)"""
    blobs = G(code)
    out_img = blobs['deconv0']  # get raw output image from GAN
    clamp_out_img = torch.clamp(out_img + BGR_mean, 0, 255) / 255
    vis_img = clamp_out_img[:, [2, 1, 0], :, :] # still use BCHW sequence 
    return vis_img

In [ ]:
class FilterVisualizerGAN():
    def __init__(self,model):
        self.model = model
        self.G = load_generator()
        self.weights = None

    def visualize(self, sz, layer, filter, weights=None, 
                  lr=0.1, opt_steps=20, blur=None, print_losses=False): #upscaling_steps=12, upscaling_factor=1.2, 
        '''Add weights to support visualize combination of channels'''
        if weights is not None:
            assert len(weights) == len(filter)
            self.weights = torch.tensor(weights,dtype=torch.float,device='cuda')
        
        activations = SaveFeatures(layer)  # register hook
        feat = 0.01 * np.random.rand(1, 4096)
        feat = torch.from_numpy(np.float32(feat))
        feat = Variable(feat, requires_grad = True).cuda()
        img = visualize_for_torchnet(self.G, feat)   
        resz_img = F.interpolate(img, (sz, sz), mode='bilinear', align_corners=True)
        #  img = (np.random.random((sz,sz, 3)) * 20 + 128.)/255 # value b/t 0 and 1     
        # img_tensor = val_tfms(resz_img) 
        img_tensor = normalize(resz_img.squeeze()).unsqueeze(0)
#         img_tensor = img_tensor.cuda()
#         img_tensor.requires_grad_();
        optimizer = optim.SGD([feat], lr=0.05,momentum=0.3,dampening=0.1)
        if not img_tensor.grad is None:
            img_tensor.grad.zero_(); 

        for n in range(opt_steps_):  # optimize pixel values for opt_steps times
            optimizer.zero_grad()
            img = visualize_for_torchnet(self.G, feat)   
            resz_img = F.interpolate(img, (sz, sz), mode='bilinear', align_corners=True)
            img_tensor = normalize(resz_img.squeeze()).unsqueeze(0)
            _=self.model(img_tensor)
            if weights is None:
                loss = -1*activations.features[0, filter].mean()
            else: 
                loss = -1*torch.einsum("ijk,i->jk", activations.features[0, filter], self.weights).mean()
            if print_losses:
                if n%5==0:
                    print(f'{n} - {float(-loss)}')
            loss.backward()
            optimizer.step()

        # convert tensor back to np
        img = val_detfms(img_tensor)
        self.output = img
        # sz = int(upscaling_factor * sz)  # calculate new image size
#             print(f'Upscale img to: {sz}')
        img = cv2.resize(img, (sz, sz), interpolation = cv2.INTER_CUBIC)  # scale image up
        if blur is not None: img = cv2.blur(img,(blur,blur))  # blur image to reduce high frequency patterns
                
        activations.close()
        return np.clip(self.output, 0, 1)
    
    def get_transformed_img(self,img,sz):
        '''
        Scale up/down img to sz. Channel last (same as input)
        image: np.array [sz,sz,3], already divided by 255"    
        '''
        return cv2.resize(img, (sz, sz), interpolation = cv2.INTER_CUBIC)
    
#     def most_activated(self, img, layer):
#         '''
#         image: np.array [sz,sz,3], already divided by 255"    
#         '''
#         img = cv2.resize(img, (224,224), interpolation = cv2.INTER_CUBIC)
#         activations = SaveFeatures(layer)
#         img_tensor = val_tfms(img)#,np.float32)
#         img_tensor = img_tensor.cuda()
        
#         _=self.model(img_tensor)
#         mean_act = [np.squeeze(to_np(activations.features[0,i].mean())) for i in range(activations.features.shape[1])]
#         activations.close()
#         return mean_act

In [ ]:
from torch_net_utils import visualize, load_generator

In [ ]:
feat = alexnet.features.cuda().eval()
FVisG = FilterVisualizerGAN(feat)

In [ ]:
img = FVisG.visualize(sz=227, layer=feat[8], filter=[1,5,3,10], weights=[1,3,1,7], blur=10, opt_steps=20, upscaling_steps=3, upscaling_factor=1.2, print_losses=True)
plt.figure(figsize=[8,8])
plt.imshow(FVis.output)
plt.show()